In [9]:
import os,sys,inspect
import json
import torch
currentdir = os.path.dirname(os.getcwd())
print(currentdir)
# parent_dir = os.path.abspath(os.path.join(currentdir, os.pardir))
root = os.path.dirname(currentdir)
print("root Folder Absoloute path: ", root)


import sys
sys.path.append(root)

import logging

logging.basicConfig(level=logging.INFO)

from IPython.display import Image

import __main__

__main__.__file__="main.py"

/home/hfaghihi/Framework/fix/DomiKnowS/examples
root Folder Absoloute path:  /home/hfaghihi/Framework/fix/DomiKnowS


In [37]:
with open("data/tracker.log", "r") as file:
    tracker = []
    for line in file:
        tracker.append(json.loads(line))
print(len(tracker))
print(tracker[0].keys())
state2idx = {'O_C': 0, 'O_D': 1, 'E': 2, 'M': 3, 'C': 4, 'D': 5}

54
dict_keys(['para_id', 'location_cand', 'states', 'loc_spans', 'boundaries', 'states_mapping', 'states_in_tokens', 'spans', 'roberta_tokens', 'states_annotation', 'roberta_bounds', 'participants', 'sentence_paragraph', 'sentence_texts', 'trips_annotation', 'actions', 'ltype', 'location_entity_cands', 'verb_mentions', 'entity_mentions', 'computed_boundaries', 'tags', 'location_indexes', 'net_results'])


In [16]:
import torch

torch.tensor(tracker[10]['net_results']['tag_logits']).shape

torch.Size([2, 5, 6])

In [36]:
a = torch.randn(5, 3, 3).softmax(dim=-1)
b = a.clone()
print(a[:, :, 0], '\n', a[:, :, 1])
print((a[:, :, 0] + a[:, :, 1]))
b[:, :, 0] = (a[:, :, 0] + a[:, :, 1])
print(b[:, :, 0])
assert b[3, 1, 0] == a[3, 1, 0] + a[3, 1, 1]
b = torch.cat((b[:, :, 0], b[:, :, ]))

tensor([[0.3347, 0.8814, 0.2403],
        [0.4371, 0.8158, 0.1250],
        [0.3785, 0.1702, 0.5897],
        [0.1611, 0.8374, 0.3235],
        [0.3536, 0.3471, 0.3818]]) 
 tensor([[0.0824, 0.0917, 0.0854],
        [0.1571, 0.1496, 0.6057],
        [0.2974, 0.7016, 0.2546],
        [0.5233, 0.1138, 0.2976],
        [0.3025, 0.2466, 0.4228]])
tensor([[0.4170, 0.9732, 0.3257],
        [0.5942, 0.9654, 0.7308],
        [0.6759, 0.8719, 0.8443],
        [0.6844, 0.9511, 0.6211],
        [0.6561, 0.5937, 0.8046]])
tensor([[0.4170, 0.9732, 0.3257],
        [0.5942, 0.9654, 0.7308],
        [0.6759, 0.8719, 0.8443],
        [0.6844, 0.9511, 0.6211],
        [0.6561, 0.5937, 0.8046]])


In [17]:
# action_probs = torch.tensor(
#     [tracker[0]['net_results']['tag_logits'] for i in range(len(tracker[0]['sentence_texts']))]
# ).squeeze(-2)
# print(action_probs.shape)
# print(action_probs.softmax(dim=-1))
# print(len(tracker[0]['sentence_texts']))

KeyError: 0

In [42]:
import json
import torch

with open("data/train_propara_roberta_version_trips.json", "r") as file:
    data = json.load(file)
print(len(data))
updated_data = []
lines = 0
for item in tracker:
    lines += len(item["sentence_texts"])
    try:
        temp = item.copy()
        new_state = []
        location_types = []
        for state_id, states in enumerate(temp['states']):
            temp2 = ["-", ]
            
            ltemp = []
            if states[0] == "?":
                ltemp.append("Unknown")
            elif states[0] == "-":
                ltemp.append("-")
            else:
                ltemp.append("Known")
                
            prev_loc = states[0]
            for loc in states[1:]:
                if loc == "?":
                    ltemp.append("Unknown")
                elif loc == "-":
                    ltemp.append("-")
                else:
                    ltemp.append("Known")

                if loc == prev_loc:
                    temp2.append("No change")
                elif loc != "-" and prev_loc == "-":
                    temp2.append("Create")
                elif loc == "-" and prev_loc != "-":
                    temp2.append("Destroy")
                else:
                    temp2.append("Move")
                prev_loc = loc
            new_state.append(temp2)
            location_types.append(ltemp)
        temp['actions'] = new_state
        temp['ltype'] = location_types
        

#         new_annotation = []
#         for state in temp['states_annotation']:
#             temp2 = state
#             for loc in temp2:
#                 if loc[0] == "?":
#                     loc[2] = item['boundaries'][-1][1] + 1
#                     loc[3] = item['boundaries'][-1][1] + 1
#             new_annotation.append(temp2)

#         temp['new_annotation'] = new_annotation
    except:
        print(item)
        raise
    updated_data.append(temp)
    
for item in updated_data:
    steps = len(item['sentence_paragraph'])
    entities = len(item['participants'])
    probs = torch.tensor(item['net_results']['tag_logits']).softmax(dim=-1)
    probs[:, :, 0] = probs[:, :, 0] + probs[:, :, 1] + probs[:, :, 2]
    probs = torch.cat((probs[:, :, 0:1], probs[:, :, 3:]), dim=-1)
    item['action_probs'] = probs.tolist()
#     tprobs = torch.softmax(torch.randn(steps, entities, 4), dim=-1)
#     item['Taction_probs'] = tprobs.tolist()
    
with open("data/train.json", "w") as file:
    json.dump(updated_data, file)

391


In [43]:
with open("data/train.json", "r") as file:
    data = json.load(file)
print(len(data))

54


In [21]:
from regr.data.reader import RegrReader
import torch


class ProparaReader(RegrReader):
    def getProcedureIDval(self, item):
        return [item["para_id"]]

    def getEntitiesval(self, item):
        return [' '.join(item['participants'])]
        
    def getEntityval(self, item):
        return item["participants"]
    
    def getContextval(self, item):
        return [item['sentence_paragraph']]

    def getStepval(self, item):
        sentences = item["sentence_texts"]
        return  sentences
    
    def getActionval(self, item):
        return torch.tensor(item['action_probs'])
    
    def getbeforeval(self, item):
        b1s = []
        b2s = []
#         for step in range(len(item["sentence_texts"])):
#             b1 = torch.zeros(len(item["sentence_texts"]))
#             b1[step] = 1
#             for step1 in range(len(item["sentence_texts"])):
#                 b2 = torch.zeros(len(item["sentence_texts"]))
#                 b2[step1] = 1
#                 b1s.append(b1)
#                 b2s.append(b2)
#         return torch.stack(b1s), torch.stack(b2s)
        for step in range(len(item['sentence_texts'])):
            for step1 in range(len(item['sentence_texts'])):
                if step < step1:
                    b1 = torch.zeros(len(item["sentence_texts"]))
                    b1[step] = 1
                    b2 = torch.zeros(len(item["sentence_texts"]))
                    b2[step1] = 1
                    b1s.append(b1)
                    b2s.append(b2)
                    
                    
        return torch.stack(b1s), torch.stack(b2s)
        

    def getbefore_trueval(self, item):
        num_steps = len(item["sentence_texts"])
        values = torch.zeros(num_steps * num_steps)
        for step in range(len(item["sentence_texts"])):
            for step1 in range(step + 1, len(item["sentence_texts"])):
                values[(step * num_steps) + step1] = 1
        return values
    
#     def getTActionval(self, item):
#         actions = []
#         for step, step_text in enumerate(item['sentence_texts']):
#             actions.append([])
#             for eid, entity in enumerate(item['participants']):
#                 actions[-1].append(item['Taction_probs'])

#         return torch.tensor(actions)


In [23]:
train_reader = ProparaReader("data/train.json")
runner = iter(train_reader)
sample = next(runner)
print(sample['before'][0].shape)
print(sample['before'][1].shape)
print(sample['before_true'].shape)
print(len(sample['Step']))
print(sample['before'])
assert sample['before'][0].shape == sample['before'][1].shape

torch.Size([15, 6])
torch.Size([15, 6])
torch.Size([36])
6
(tensor([[1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.]]), tensor([[0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0.,

In [121]:
from regr.graph import Graph, Concept, Relation
from regr.graph.logicalConstrain import orL, andL, existsL, notL, atLeastL, atMostL, ifL, nandL, V, exactL
from regr.graph import EnumConcept


Graph.clear()
Concept.clear()
Relation.clear()

with Graph('global') as graph:
    procedure = Concept("procedure")
    
    context = Concept("context")
    entities = Concept("entities")
    
    (procedure_context, procedure_entities) = procedure.has_a(context, entities)
    
    entity = Concept('entity')
    (entity_rel, ) = entities.contains(entity)
    
    step = Concept("step")
    (context_step, ) = context.contains(step)
    
    before = Concept("before")
    (before_arg1, before_arg2) = before.has_a(arg1=step, arg2=step)
    
    action = Concept(name='action')
    (action_step, action_entity) = action.has_a(step, entity)
    
    
    action_label = action(name="action_label", ConceptClass=EnumConcept, values=["nochange", "move", "create", "destroy"])
    
    ifL(
        # action a1 is destroy, i is a1's step and e is action entity
        andL(
            action_label.destroy('a1'), 
            step('i', path=('a1', action_step)),
            entity('e', path=('a1', action_entity))
            ), 
        # then either
        orL(
            # step j associated with entity e, which is before step i cannot be associated with destroy action a2
            ifL(
                step('j', path=(('i', before_arg1.reversed, before_arg2))), 
                notL(action_label.destroy('a2', path=(('j', action_step.reversed), ('e', action_entity.reversed))))
                ), 
            # or if  
            ifL(
                # step j1 which is before step i is associated with destroy action a2
                andL(
                    step('j1', path=('i', before_arg1.reversed, before_arg2)), 
                    action_label.destroy('a2', path=(('j1', action_step.reversed), ('e', action_entity.reversed)))
                    ), 
                # then exists step k associated with entity e, which is between step i and j1 associated with create action a3
                existsL(
                    andL(
                        step('k', path=(('j1', before_arg2.reversed, before_arg1), ('i', before_arg1.reversed, before_arg2))), 
                        action_label.create('a3', path=(('k', action_step.reversed), ('e', action_entity.reversed)))
                        )
                    )
                )
            ), active = True
        ) 

    # ----------------------------
    
    ifL(
        # action a1 is create, i is a1's step and e is action entity
        andL(
            action_label.create('a1'), 
            step('i', path=('a1', action_step)),
            entity('e', path=('a1', action_entity))
            ), 
        # then either
        orL(
            # step j associated with entity e, which is before step i cannot be associated with create action a2
            ifL(
                step('j', path=(('i', before_arg1.reversed, before_arg2))), 
                notL(action_label.create('a2', path=(('j', action_step.reversed), ('e', action_entity.reversed))))
                ), 
            # or if  
            ifL(
                # step j1 which is before step i is associated with create action a2
                andL(
                    step('j1', path=('i', before_arg1.reversed, before_arg2)), 
                    action_label.create('a2', path=(('j1', action_step.reversed), ('e', action_entity.reversed)))
                    ), 
                # then exists step k associated with entity e, which is between step i and j1 associated with destroy action a3
                existsL(
                    andL(
                        step('k', path=(('j1', before_arg2.reversed, before_arg1), ('i', before_arg1.reversed, before_arg2))), 
                        action_label.destroy('a3', path=(('k', action_step.reversed), ('e', action_entity.reversed)))
                        )
                    )
                )
            ), active = True
        )
    
    #  ------------
    
    ifL(
        # action a1 is move, i is a1's step and e is action entity
        andL(
            action_label.move('a1'), 
            step('i', path=('a1', action_step)),
            entity('e', path=('a1', action_entity))
            ), 
        # then either
        orL(
            # step j associated with entity e, which is before step i cannot be associated with destroy action a2
            ifL(
                step('j', path=(('i', before_arg1.reversed, before_arg2))), 
                notL(action_label.destroy('a2', path=(('j', action_step.reversed), ('e', action_entity.reversed))))
                ), 
            # or if  
            ifL(
                # step j1 which is before step i is associated with destroy action a2
                andL(
                    step('j1', path=('i', before_arg1.reversed, before_arg2)), 
                    action_label.destroy('a2', path=(('j1', action_step.reversed), ('e', action_entity.reversed)))
                    ), 
                # then exists step k associated with entity e, which is between step i and j1 associated with create action a3
                existsL(
                    andL(
                        step('k', path=(('j1', before_arg2.reversed, before_arg1), ('i', before_arg1.reversed, before_arg2))), 
                        action_label.create('a3', path=(('k', action_step.reversed), ('e', action_entity.reversed)))
                        )
                    )
                )
            ), active = True
        )
    
    # if 
#     ifL(
#         # action a1 is destroy, i is a1's step and e is action entity
#         andL(
#             action_label.destroy('a1'), 
#             step('i', path=('a1', action_step)),
#             entity('e', path=('a1', action_entity))
#             ), 
        
#         # step j associated with entity e, which is before step i cannot be associated with destroy action a2
#         andL(
#             step('j', path=(('e', action_entity.reversed, action_step), ('i', before_arg1.reversed, before_arg2))), 
#             notL(action_label.destroy('a2', path=('j', action_step.reversed)))
#             ), 
#         active = False
#         ) 




main.py:25: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'step'), ('arg2', 'step')]) is used.
  before_arg1,


In [122]:
from regr.sensor.pytorch.sensors import FunctionalSensor, JointSensor, ReaderSensor, FunctionalReaderSensor
from regr.sensor.pytorch.learners import ModuleLearner
from regr.sensor.pytorch.relation_sensors import CompositionCandidateSensor
from regr.sensor.pytorch.query_sensor import DataNodeReaderSensor

class JointFunctionalReaderSensor(JointSensor, FunctionalReaderSensor):
    pass


procedure['id'] = ReaderSensor(keyword='ProcedureID')
context['text'] = ReaderSensor(keyword='Context')
entities['text'] = ReaderSensor(keyword='Entities')

def make_procedure(arg1m, arg2m, data):
    total_procedures = len(arg1m) * len(arg2m)
    rel_links1 = torch.zeros(total_procedures, len(arg1m))
    rel_links2 = torch.zeros(total_procedures, len(arg2m))
    past1 = 0
    past2 = 0
    for i in range(total_procedures):
        rel_links1[i, past2: past2 + len(arg2m)] = 1
        past2 = past2 + len(arg2m)
        rel_links2[i, past1: past1 + len(arg1m)] = 1
        past1 = past1 + len(arg1m)

    return rel_links1, rel_links2
    
    
procedure[procedure_context.reversed, procedure_entities.reversed] = JointFunctionalReaderSensor(context['text'], entities['text'], keyword="ProcedureID", forward=make_procedure)

def read_initials(*prev, data):
    number = len(data)
    rel_links = torch.ones(number, 1)
    indexes = [i for i in range(number)]
#     print(rel_links, data, indexes)
    return rel_links, data, indexes

entity[entity_rel, 'text', 'index'] = JointFunctionalReaderSensor(entities['text'], keyword='Entity', forward=read_initials)

step[context_step, 'text', 'index'] = JointFunctionalReaderSensor(context['text'], keyword='Step', forward=read_initials)

def make_actions(r1, r2, entities, steps):
#     print(r1, r2)
    all_actions = len(steps) * len(entities)
    link1 = torch.zeros(all_actions, len(steps))
    link2 = torch.zeros(all_actions, len(entities))
    for i in range(len(entities)):
        link2[i*len(steps):(i+1)*len(steps),i] = 1

    for j in range(all_actions):
        link1[j, j%len(steps)] = 1
    
#     print(link1, link2)
#     print(link1.shape, link2.shape)
#     print("steps: ", len(steps))
#     print("entities: ", len(entities))
    return link1, link2
    
action[action_step.reversed, action_entity.reversed] = JointSensor(entity[entity_rel], step[context_step], entity['index'], step['index'], forward=make_actions)

def read_labels(*prevs, data):
#     print(prevs[0].shape, prevs[1].shape)
#     print(data.shape)
    c = data.view(data.shape[0] * data.shape[1], data.shape[2])
#     print(c.shape)
    return c
#     pass
    
action[action_label] = FunctionalReaderSensor(action_step.reversed, action_entity.reversed, keyword="Action", forward=read_labels)


In [123]:
from regr.program import POIProgram, IMLProgram, SolverPOIProgram
from regr.program.primaldualprogram import PrimalDualProgram
from regr.program.model.pytorch import SolverModel
from regr.program.metric import MacroAverageTracker, PRF1Tracker, PRF1Tracker, DatanodeCMMetric
from regr.program.loss import NBCrossEntropyLoss, NBCrossEntropyIMLoss

In [124]:
program = SolverPOIProgram(graph, poi=(procedure, action, action_label), inferTypes=['ILP', 'local/argmax'], loss=MacroAverageTracker(NBCrossEntropyLoss()), metric={'ILP':PRF1Tracker(DatanodeCMMetric()),'argmax':PRF1Tracker(DatanodeCMMetric('local/argmax'))})
# program = POIProgram(graph, poi=(procedure, action, action_label), loss=MacroAverageTracker(NBCrossEntropyLoss()), metric={'argmax':PRF1Tracker(DatanodeCMMetric('local/argmax'))})
program = SolverPOIProgram(graph, poi=(procedure, action, action_label), inferTypes=['local/argmax'], loss=MacroAverageTracker(NBCrossEntropyLoss()), metric={'ILP':PRF1Tracker(DatanodeCMMetric()),'argmax':PRF1Tracker(DatanodeCMMetric('local/argmax'))})



In [125]:
lbp = program

dataset = ProparaReader(file="data/train.json")  # Adding the info on the reader

#     lbp.test(dataset, device='auto')
all_updates = []
for datanode in lbp.populate(dataset, device="cpu"):
#     tdatanode = datanode.findDatanodes(select = context)[0]
#     print(len(datanode.findDatanodes(select = context)))
#     print(tdatanode.getChildDataNodes(conceptName=step))
    datanode.inferILPResults(action, fun=None)
    final_output = {
        "id": datanode.getAttribute("id"),
        "steps": [],
        "actions": [],
        "steps_before": [],
        "actions_before": [],
    }

    entities = datanode.findDatanodes(select=entity)
#     print(len(entities))
    steps = datanode.findDatanodes(select=step)
#     print(len(steps), "\n")
    for action_info in datanode.findDatanodes(select=action):
        c = action_info.getAttribute(action_label, "ILP")
        final_output["actions"].append(c)
        c = action_info.getAttribute(action_label)
        final_output["actions_before"].append(c)

    final_output['actions'] = torch.stack(final_output['actions'])
    final_output['actions'] = final_output['actions'].view(len(entities), len(steps), 4)
    
    final_output['actions_before'] = torch.stack(final_output['actions_before'])
    final_output['actions_before'] = final_output['actions_before'].view(len(entities), len(steps), 4)
    
    all_updates.append(final_output)

In [126]:
all_updates[26]['actions']

tensor([[[1., 0., 0., 0.],
         [0., 0., 0., 1.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 1., 0.]],

        [[1., 0., 0., 0.],
         [0., 0., 1., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.]],

        [[1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 1., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.]],

        [[1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 1., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.]],

        [[1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 1., 0.],
         [1., 0., 0., 0.]]])

In [127]:
all_updates[26]['actions_before'].argmax(dim=-1)

tensor([[0, 3, 0, 0, 0, 2],
        [0, 2, 0, 0, 0, 0],
        [0, 0, 2, 0, 0, 0],
        [0, 0, 0, 2, 0, 0],
        [0, 0, 0, 0, 2, 0]])